In [1]:
def addLists(*lists):
    return [sum(tup) for tup in zip(*lists)]

def multiplyLists(*lists):
    return [reduce(operator.mul, tup, 1) for tup in zip(*lists)]

In [2]:
class Blueprint:
    def __init__(self, oreRobotOreCost, clayRobotOreCost, obsidianRobotOreCost, obsidianRobotClayCost, geodeRobotObsidianCost, geodeRobotOreCost):
        self.oreRobotOreCost = oreRobotOreCost
        self.clayRobotOreCost = clayRobotOreCost
        self.obsidianRobotOreCost = obsidianRobotOreCost
        self.obsidianRobotClayCost = obsidianRobotClayCost
        self.geodeRobotObsidianCost = geodeRobotObsidianCost
        self.geodeRobotOreCost = geodeRobotOreCost

        self.oreRobotCostVector = [-oreRobotOreCost, 0,0,0]
        self.clayRobotCostVector = [-clayRobotOreCost, 0,0,0]
        self.obsidianRobotCostVector = [-obsidianRobotOreCost, -obsidianRobotClayCost,0,0]
        self.geodeRobotCostVector = [-geodeRobotOreCost, 0,-geodeRobotObsidianCost,0]
    
    def __str__(self):
        return f'{self.oreRobotOreCost}, {self.clayRobotOreCost}, {self.obsidianRobotOreCost}, {self.obsidianRobotClayCost}, {self.geodeRobotObsidianCost}, {self.geodeRobotOreCost}'

In [3]:
class DecisionPath:
    def __init__(self, resources, robots, round):
        self.resources = resources
        self.robots = robots
        self.round = round
        self.value = self.__calculateValue__()
        
    def __calculateValue__(self):
        weightVector = [1,2,3,4]
        robotWeight = 3
        return sum(multiplyLists(self.resources, weightVector))+ sum(multiplyLists(self.robots,weightVector, [robotWeight,robotWeight,robotWeight,robotWeight]))
        

    def __str__(self):
        return f'RES: {self.resources}, ROBOTS: {self.robots}, ROUND: {self.round}, VALUE: {self.value}'

In [4]:
from functools import reduce
import operator


with open("test-input.txt") as f:
    lines = list(map(lambda x: x.strip(), f.readlines()))

blueprints = []

for line in lines:
    sentences = line.split(".")
    oreRobotOreCost  = int(sentences[0].split(" ")[-2]) 
    clayRobotOreCost  = int(sentences[1].split(" ")[-2])
    obsidianRobotOreCost  = int(sentences[2].split(" ")[-5])
    obsidianRobotClayCost  = int(sentences[2].split(" ")[-2])
    geodeRobotOreCost  = int(sentences[3].split(" ")[-5])
    geodeRobotObsidianCost  = int(sentences[3].split(" ")[-2])

    blueprints+= [Blueprint(oreRobotOreCost,
    clayRobotOreCost,
    obsidianRobotOreCost,
    obsidianRobotClayCost,
    geodeRobotObsidianCost,
    geodeRobotOreCost)]

In [5]:
def getOreRobotPath(blueprint: Blueprint, path:DecisionPath):
    resources = addLists(blueprint.oreRobotCostVector, path.resources, path.robots)
    robots = addLists([1,0,0,0], path.robots)
    return DecisionPath(resources, robots, path.round+1)

def getClayRobotPath(blueprint: Blueprint, path:DecisionPath):
    resources = addLists(blueprint.clayRobotCostVector, path.resources, path.robots)
    robots = addLists([0,1,0,0], path.robots)
    return DecisionPath(resources, robots, path.round+1)

def getObsidianRobotPath(blueprint: Blueprint, path:DecisionPath):
    resources = addLists(blueprint.obsidianRobotCostVector, path.resources, path.robots)
    robots = addLists([0,0,1,0], path.robots)
    return DecisionPath(resources, robots, path.round+1)

def getGeodeRobotPath(blueprint: Blueprint, path:DecisionPath):
    resources = addLists(blueprint.geodeRobotCostVector, path.resources, path.robots)
    robots = addLists([0,0,0,1], path.robots)
    return DecisionPath(resources, robots, path.round+1)

def getNoActionPath(blueprint: Blueprint, path:DecisionPath):
    resources = addLists(path.resources, path.robots)
    return DecisionPath(resources, path.robots, path.round+1)

def getPossiblePaths(path:DecisionPath, blueprint:Blueprint):
    result = [getNoActionPath(blueprint, path)]
    if len(list(filter(lambda x: x<0, addLists(path.resources, blueprint.oreRobotCostVector)))) == 0:
        result.append(getOreRobotPath(blueprint, path))
    if len(list(filter(lambda x: x<0, addLists(path.resources, blueprint.clayRobotCostVector)))) == 0:
        result.append(getClayRobotPath(blueprint, path))
    if len(list(filter(lambda x: x<0, addLists(path.resources, blueprint.obsidianRobotCostVector)))) == 0:
        result.append(getObsidianRobotPath(blueprint, path))
    if len(list(filter(lambda x: x<0, addLists(path.resources, blueprint.geodeRobotCostVector)))) == 0:
        result.append(getGeodeRobotPath(blueprint, path))
    return result

def getMaximumPossibleGeodes(path: DecisionPath):
    return path.resources[3]+path.robots[3]+sum(range(0,24-path.round))

bestPath = DecisionPath([0,0,0,0], [0,0,0,1], 0)
start = DecisionPath([0,0,0,0], [1,0,0,0], 0)
maxStackSize = 50

for blueprint in blueprints[1:2]:
    stack = [start]
    while len(stack) > 0:
        currentPath = stack.pop(0)
        if currentPath.round < 24:
            newPaths = getPossiblePaths(currentPath, blueprint)
            for newPath in newPaths:
                if newPath.round<24:
                    if newPath.round > 20:
                        maximumPossibleGeodes = getMaximumPossibleGeodes(newPath)
                        if maximumPossibleGeodes > bestPath.resources[3]:
                            stack.append(newPath)
                    else: stack.append(newPath)
                elif newPath.resources[3] > bestPath.resources[3]:
                    bestPath = newPath
                    print(bestPath, len(stack))
            stack.sort(key=lambda x: x.value, reverse=True)
            if len(stack) > maxStackSize:
                stack = stack[0:maxStackSize]

bestPath.resources[3]

RES: [23, 102, 8, 1], ROBOTS: [4, 12, 4, 1], ROUND: 24, VALUE: 387 35
RES: [23, 97, 12, 2], ROBOTS: [4, 12, 4, 1], ROUND: 24, VALUE: 393 35
RES: [23, 92, 16, 3], ROBOTS: [4, 12, 4, 1], ROUND: 24, VALUE: 399 35
RES: [23, 87, 20, 4], ROBOTS: [4, 12, 4, 1], ROUND: 24, VALUE: 405 36
RES: [23, 86, 8, 5], ROBOTS: [4, 11, 4, 2], ROUND: 24, VALUE: 377 32
RES: [23, 80, 12, 7], ROBOTS: [4, 11, 4, 2], ROUND: 24, VALUE: 385 32
RES: [23, 74, 16, 9], ROBOTS: [4, 11, 4, 2], ROUND: 24, VALUE: 393 30
